# SFT 監督式微調

## 什麼是 SFT (Supervised Fine-Tuning)?

Supervised Fine-Tuning (SFT) 是最直接的微調方法,使用標註好的「輸入-輸出」對來訓練模型。

## SFT 的優勢與限制
**優勢:**
- 概念簡單,易於理解和實現
- 資料需求相對明確
- 訓練穩定,收斂快
- 可以快速適應特定任務
**限制:**
- 需要大量高質量標註資料
- 可能過擬合訓練資料
- 對資料質量極為敏感

## 資料格式設計

Alpaca 格式（單輪對話）：

In [ ]:
{
  "instruction": "將以下英文翻譯成中文",
  "input": "The weather is beautiful today.",
  "output": "今天天氣很好。"
}

Chat 格式（多輪對話）：

In [ ]:
{
  "conversations": [
    {"role": "user", "content": "什麼是機器學習？"},
    {"role": "assistant", "content": "機器學習是人工智慧的一個分支..."},
    {"role": "user", "content": "能舉個例子嗎？"},
    {"role": "assistant", "content": "當然！例如垃圾郵件過濾..."}
  ]
}

實際使用時的 Prompt 模板：

In [ ]:
<|system|>你是一個有幫助的 AI 助手</s>
<|user|>什麼是機器學習？</s>
<|assistant|>機器學習是人工智慧的一個分支...</s>
<|user|>能舉個例子嗎？</s>
<|assistant|>

模型學習在 <|assistant|> 標記後生成回應。

### to be continued